<a href="https://colab.research.google.com/github/aekanun2020/2022-PEA-ML-sklearn-Spark-/blob/main/L_sckitlearn_Regression_Pipeline_Estimation_of_Arrival_Delay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
! wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2022-11-08 22:03:28--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.48.94
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.48.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M  46.6MB/s    in 15s     

2022-11-08 22:03:43 (43.9 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [3]:
! wc -l 2008.csv

7009729 2008.csv


In [4]:
f = "2008.csv"
num_lines = sum(1 for l in open(f))
size = int(num_lines / 100)
import random
skip_idx = random.sample(range(1, num_lines), num_lines - size)
raw_pd = pd.read_csv(f, skiprows=skip_idx)

In [5]:
#raw_pd = pd.read_csv("2008.csv")

In [6]:
raw_pd.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,NaN,700,NaN,830,WN,126,...,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,1704.0,1645,1929.0,1930,WN,340,...,7.0,9.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,1143.0,1035,1621.0,1505,WN,574,...,3.0,29.0,0,NaN,0,0.0,0.0,14.0,0.0,62.0
3,2008,1,3,4,1419.0,1400,1534.0,1515,WN,420,...,4.0,6.0,0,NaN,0,12.0,0.0,0.0,0.0,7.0
4,2008,1,3,4,1825.0,1745,1921.0,1850,WN,555,...,4.0,5.0,0,NaN,0,4.0,0.0,0.0,0.0,27.0


In [7]:
raw_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70096 entries, 0 to 70095
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               70096 non-null  int64  
 1   Month              70096 non-null  int64  
 2   DayofMonth         70096 non-null  int64  
 3   DayOfWeek          70096 non-null  int64  
 4   DepTime            68726 non-null  float64
 5   CRSDepTime         70096 non-null  int64  
 6   ArrTime            68564 non-null  float64
 7   CRSArrTime         70096 non-null  int64  
 8   UniqueCarrier      70096 non-null  object 
 9   FlightNum          70096 non-null  int64  
 10  TailNum            69249 non-null  object 
 11  ActualElapsedTime  68539 non-null  float64
 12  CRSElapsedTime     70085 non-null  float64
 13  AirTime            68539 non-null  float64
 14  ArrDelay           68539 non-null  float64
 15  DepDelay           68726 non-null  float64
 16  Origin             700

In [8]:
def t_timeperiod(origin):
    if origin is None:
        period = None
    elif origin > 0 and origin < 600:
        period = '00.01-05.59'
    elif origin >= 600 and origin <=1200:
        period = '06.00-11.59'
    elif origin >= 1200 and origin <= 1800:
        period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
        period = '18.00-24.00'
    else:
        period = 'NA'
    return period

In [9]:
raw_pd['DepTime'] = raw_pd['DepTime'].apply(lambda x:t_timeperiod(x))

In [10]:
max_distance = raw_pd['Distance'].max()
min_distance = raw_pd['Distance'].min()

In [11]:
max_ArrDelay = raw_pd['ArrDelay'].max()
min_ArrDelay = raw_pd['ArrDelay'].min()

In [12]:
def t_normalized_distance(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_distance)/(max_distance-min_distance))

In [13]:
def t_normalized_ArrDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [14]:
raw_pd['Distance'] = raw_pd['Distance'].apply(lambda x:t_normalized_distance(x))

In [15]:
raw_pd['ArrDelay'] = raw_pd['ArrDelay'].apply(lambda x:t_normalized_ArrDelay(x))

In [16]:
features_pd = raw_pd[['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay']]

In [17]:
final_pd = features_pd.dropna()

In [18]:
final_pd.count()

UniqueCarrier    68539
Origin           68539
Dest             68539
DepTime          68539
TaxiOut          68539
TaxiIn           68539
DepDelay         68539
DayOfWeek        68539
Distance         68539
ArrDelay         68539
dtype: int64

In [19]:
features_pd.head(20)

,UniqueCarrier,Origin,Dest,DepTime,TaxiOut,TaxiIn,DepDelay,DayOfWeek,Distance,ArrDelay
0,WN,LAS,OAK,NA,NaN,NaN,NaN,4,0.076252,NaN
1,WN,LAS,SEA,12.00-17.59,9.0,7.0,19.0,4,0.169337,0.052960
2,WN,LAS,TUL,06.00-11.59,29.0,3.0,68.0,4,0.211925,0.112928
3,WN,LAX,OAK,12.00-17.59,6.0,4.0,19.0,4,0.062056,0.068536
4,WN,LBB,AUS,18.00-24.00,5.0,4.0,40.0,4,0.062868,0.077882
5,WN,MDW,BUF,18.00-24.00,16.0,3.0,62.0,4,0.088623,0.105140
6,WN,MDW,BWI,06.00-11.59,10.0,3.0,14.0,4,0.117623,0.061526
7,WN,MDW,MCI,06.00-11.59,6.0,5.0,2.0,4,0.075847,0.047508
8,WN,MDW,SFO,12.00-17.59,25.0,4.0,209.0,4,0.369905,0.221184
9,WN,OAK,SAN,12.00-17.59,10.0,4.0,51.0,4,0.084161,0.088006


In [20]:


#raw_pd.isnull().sum()
#nullCol_series = raw_pd.isnull().sum()
#notNull_list = list(nullCol_series[nullCol_series == 0].index)
#notNull_list
#requiredCol_list = ['TaxiIn','TaxiOut','ArrDelay']
#selectedCol_list = notNull_list + requiredCol_list
#noNull_pd = raw_pd.loc[:,selectedCol_list].dropna()
#noNull_pd.isnull().sum()
#noNull_pd.describe()
#noNull_pd.info()

In [21]:
#final_pd = noNull_pd

In [22]:
import pandas as pd
from seaborn import load_dataset
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [23]:
train_df = final_pd.sample(frac=0.8,random_state=123)

In [24]:
test_df = final_pd.drop(train_df.index)

In [25]:
train_df.count()

UniqueCarrier    54831
Origin           54831
Dest             54831
DepTime          54831
TaxiOut          54831
TaxiIn           54831
DepDelay         54831
DayOfWeek        54831
Distance         54831
ArrDelay         54831
dtype: int64

In [26]:
test_df.count()

UniqueCarrier    13708
Origin           13708
Dest             13708
DepTime          13708
TaxiOut          13708
TaxiIn           13708
DepDelay         13708
DayOfWeek        13708
Distance         13708
ArrDelay         13708
dtype: int64

In [27]:
train_labels = train_df.filter(['ArrDelay'])
train_df = train_df.loc[:, train_df.columns != 'ArrDelay']

In [28]:
type(train_labels)

pandas.core.frame.DataFrame

In [29]:
test_labels = test_df.filter(['ArrDelay'])
test_df = test_df.loc[:, test_df.columns != 'ArrDelay']

In [30]:
X_train = train_df
X_test = test_df
y_train = train_labels
y_test = test_labels

In [31]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['UniqueCarrier', 'Origin', 'Dest', 'DepTime']
Numerical columns are: ['TaxiOut', 'TaxiIn', 'DepDelay', 'DayOfWeek', 'Distance']


In [32]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [33]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical))
                     #('imputer', SimpleImputer(strategy='median')),
                     #('scaler', MinMaxScaler())
                    ])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Model Training
pipe = Pipeline(steps=[('preprocessor', preprocessor),
              ('model', DecisionTreeRegressor(max_depth=None))])
pipeline_model = pipe.fit(X_train, y_train)

In [34]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

predictions = pipeline_model.predict(X_test)
print("\t RMSE:", mean_squared_error(y_test, predictions, squared=False))
print("\t R2 score:", r2_score(y_test, predictions))

	 RMSE: 0.011026030499800549
	 R2 score: 0.8818548183278208
